<a href="https://colab.research.google.com/github/ranielyn/gpt2-paper-title-generator/blob/master/Progress_Report_2_Scrape_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install arxivscraper
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# **Scrape Finetune Sample**


# **Step 1 - Scrape data**
In order to work, we require that the data is saved to a csv file with one piece of text (e.g. one paper title) on each line. There are no actual commas in the file, and there shouldn't be newlines contained in each piece of text.


In [2]:
import arxivscraper as ax
import numpy as np

'''
# scraper for arxiv stat.ml
scraper = ax.Scraper(category='stat', date_from='2017-08-01',
                     date_until='2019-07-01', t=10, 
                     filters={'categories':['stat.ml'],'abstract':['learning']})

# scraper for arxiv q-bio
scraper = ax.Scraper(category='q-bio', date_from='2016-08-01',
                     date_until='2019-07-01', t=10, 
                     filters={'categories':['q-bio.GN', 'q-bio.NC']})
'''

# scraper for arxiv physics
scraper = ax.Scraper(category='physics', date_from='2019-05-01',
                     date_until='2019-07-03', t=10,
                     filters={'categories':['quant-ph']})

output = scraper.scrape()



# cols = ('id', 'title', 'categories', 'abstract', 'doi', 'created', 'updated', 'authors')
titles = [' '.join(o['title'].split()) for o in output]
np.savetxt('title_ref2.txt', np.array(titles), fmt='%s')

fetching up to  1000 records...
fetching up to  2000 records...
fetching up to  3000 records...
fetching up to  4000 records...
fetching up to  5000 records...
fetching up to  6000 records...
fetching up to  7000 records...
fetching up to  8000 records...
fetching up to  9000 records...
fetching up to  10000 records...
fetching up to  11000 records...
fetching up to  12000 records...
fetching up to  13000 records...
fetching is completed in 222.0 seconds.
Total number of records 1430


# **step 2 - finetune gpt2**
this code will download gpt2 and finetune it on the file title.csv, generating samples at intermediate steps

In [3]:
import gpt_2_simple as gpt2

model_name = "124M" # "355M" for larger model (it's 1.4 GB)
gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/
gpt2.mount_gdrive()
file_name = "title_ref2.txt"
gpt2.copy_file_from_gdrive(file_name)
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=200,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=50,
              save_every=100
              )   # steps is max number of training steps

gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="quantum",
              nsamples=5,
              batch_size=5
              )

Fetching checkpoint: 1.05Mit [00:00, 243Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 959kit/s]
Fetching hparams.json: 1.05Mit [00:00, 382Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:17, 6.45Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 331Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.16Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.16Mit/s]


Mounted at /content/drive
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 220.79it/s]

Loading dataset...
dataset has 2387 tokens
Training...


[10 | 1042.24] loss=3.61 avg=3.61
[20 | 2111.78] loss=2.14 avg=2.87
[30 | 3183.70] loss=0.78 avg=2.17
[40 | 4248.46] loss=0.41 avg=1.72
[50 | 5309.16] loss=0.07 avg=1.38
======== SAMPLE 1 ========
 codes the operation of quantum bell inequalities: experiment and results The evolution of bell inequalities experimentally suggested modifications in an electro-optomechanical device		
quantum correlations and global coherence in distributed fault analysis		
generalized quantum correlations: the role of global moderators		
are quantum world systems able to process and process multiple jitterbugti?		
are quantum coherence simple?
are light-dependent knockdown signs common to quantum ensembles		
modulation effect in multiphoton pair production		
quantum optical nonlinearities induced by rydberg-rydberg interactions		
experimental demonstration of the no-hiding theorem on the 5 qubit ibm quantum computer		
a set theoretical approach in the domains of quantum computation		
quantum sphere-packin

# **Step 3 - Look at the Model**


generating new samples from the finetuned model
- generate one sample

In [5]:
prefix = 'neural' # None is default
text = gpt2.generate(sess,
              length=40,
              temperature=0.7,
              prefix=prefix,
              nsamples=1,
              batch_size=1,
              return_as_list=True
             )


t = text[0].title()
t = t.replace('<|Startoftext|>', '').replace('\n', '') # remove extraneous stuff
t = t[:t.index('<|Endoftext|>')] # only get one title
print(t)

ValueError: ignored

generate a bunch of samples

In [ ]:
text = gpt2.generate(sess,
#               length=40,
              temperature=0.7,
              prefix=None,
              nsamples=100,
              batch_size=1,
              return_as_list=True
             )


t = text[0].title()
t = t.replace('<|Startoftext|>', '').replace('\n', '') # remove extraneous stuff
t = t[:t.index('<|Endoftext|>')] # only get one title
print(t)